In [ ]:
import pandas as pd
import numpy as np
from statistics import mean
pd.set_option('display.max_rows', 100)

In [ ]:
# Raw data from Burgdorf M, Pütz T. Dokumentation der Raumvariablen des BBSR im Regionalfile. Annex 5 to the user manual for the MiD 2017. 2019.

#df = pd.read_csv('C:/Users/arning/sciebo/04_Research/02_Current projects/2023_04_PLUG-IN/04_Data BBSR/250m_slope_transformed.csv')
#df = df[['xcoord','ycoord','zcoord']]
#df.to_csv('C:/Users/arning/sciebo/04_Research/02_Current projects/2023_04_PLUG-IN/04_Data BBSR/250m_slope_transformed.csv')
#df

In [ ]:
#read 250m slope data
df = pd.read_csv('C:/Users/arning/sciebo/04_Research/02_Current projects/2023_04_PLUG-IN/04_Data BBSR/250m_slope_transformed.csv')
#df.sample(100)

In [ ]:
#round values to nearest 250m value to realign with 1km cells
df['xcoord'] = np.round(df['xcoord'] / 250) * 250
df['ycoord'] = np.round(df['ycoord'] / 250) * 250
#df.head(100)

In [ ]:
#df.to_csv('C:/Users/arning/sciebo/04_Research/02_Current projects/2023_04_PLUG-IN/04_Data BBSR/250m_slope_transformed_rounded.csv')

In [ ]:
#assign values to corresponding 1km cells
#load 1km cells (from Raumtyp)
df1km = pd.read_csv('C:/Users/arning/sciebo/04_Research/02_Current projects/2023_04_PLUG-IN/11_Mode Choice/Python/Raumtyp.csv')
#df1km = df1km.head(100)

In [ ]:
# Add coordinates (indicated south-western corner of 1km cell)
df1km['ycoord'] = 0
df1km['xcoord'] = 0

for index, row in df1km.iterrows():
    df1km.at[index, 'ycoord'] = int(row['cellname'][4:8])*1000
    df1km.at[index, 'xcoord'] = int(row['cellname'][9:13])*1000

#df1km

In [ ]:
#for every 1km cell, create list of 250m cells that should be taken into account for
#attention: coordinates of 250m cells refer to their centerpoint. We simplify by using it also as their south-western corner; effectively this shifts all data by 125m to the South and West.

def return_250m_cells_y(y):
    y_list = [y,y,y,y,y+250,y+250,y+250,y+250,y+500,y+500,y+500,y+500,y+750,y+750,y+750,y+750]
    return y_list

def return_250m_cells_x(x):
    x_list = [x,x+250,x+500,x+750,x,x+250,x+500,x+750,x,x+250,x+500,x+750,x,x+250,x+500,x+750]
    return x_list

In [ ]:
# Initialize empty lists in the 'ycoords_250' and 'xcoords_250' columns
df1km['ycoords_250'] = [None] * len(df1km)
df1km['xcoords_250'] = [None] * len(df1km)

# Iterate through the DataFrame and populate the columns
for index, row in df1km.iterrows():
    df1km.at[index, 'ycoords_250'] = return_250m_cells_y(row['ycoord'])
    df1km.at[index, 'xcoords_250'] = return_250m_cells_x(row['xcoord'])
    
#df1km

In [ ]:
#Get slope values from corresponding 250m cells onto 1km cells

#Initialize empty lists in the slope values column
df1km['250m_slopes'] = [[]] * len(df1km)
df1km

#iterrate over the 1km cells df
for index, row in df1km.iterrows():
    temp_slope_values = []
    #for every 1km cell, iterate over all 16 assigned 250m cell coordinate pairs, look up slope value in 250m df and add it to list    
    for i in range(16):
        #look up slope value of one 250m cell
        temp = df[(df.xcoord == row.xcoords_250[i]) & (df.ycoord == row.ycoords_250[i])]['zcoord']
        #add temp slope value to list of values of the 1km cell
        temp_slope_values.append(temp.values[0] if not temp.empty else None)
    df1km.at[index, '250m_slopes'] = temp_slope_values

In [ ]:
#average values for each 1km cell
#initialize column
df1km['slope']=[None] * len(df1km)

#keep slope as None if one entry of the 250m_slopes list contains None
df1km_nones = df1km[df1km['250m_slopes'].apply(lambda x: None in x)]

#create df without nones to apply the averaging function there
df1km_no_nones = df1km[~df1km['250m_slopes'].apply(lambda x: None in x)]

for index, rows in df1km_no_nones.iterrows():
    df1km_no_nones.at[index, 'slope'] = sum(df1km_no_nones.at[index, '250m_slopes'])/len(df1km_no_nones.at[index, '250m_slopes'])

In [ ]:
#Now more complicated average for those 1km cells where one or more 250m cells have non as entry

for index, row in df1km_nones.iterrows():
    non_none_values = [d for d in row['250m_slopes'] if d is not None]
    if non_none_values:
        df1km_nones.at[index, 'slope'] = mean(non_none_values)
    else:
        df1km_nones.at[index, 'slope'] = None

#print(df1km_nones)

#recombine dataframes
data = [df1km_nones, df1km_no_nones]
df1km = pd.concat(data)

In [ ]:
#df1km.to_csv('C:/Users/arning/sciebo/04_Research/02_Current projects/2023_04_PLUG-IN/11_Mode Choice/1km_Raumtyp_Slope.csv')